## Monedas

In [1]:
import QC_Financial as qcf

In [2]:
# Moneda CLP y USD
clp = qcf.QCCLP()
usd = qcf.QCUSD()
monedas = [clp, usd]

In [3]:
# Métodos: get_name, get_iso_code, get_iso_number, get_decimal_places, amount
for moneda in monedas:
    print moneda.get_name()
    print moneda.get_iso_code()
    print moneda.get_iso_number()
    print moneda.get_decimal_places()
    print moneda.amount(100.123456)
    print

Chilean Peso
CLP
152
0
100.0

U. S. Dollar
USD
840
2
100.12



## Fechas
Las fechas se representan con objetos de tipo QCDate. Para inicializar un QCDate se requiere el día, el mes y el año de la fecha.  También se puede inicializar sin valor (default constructor) en cuyo caso se obtendrá el 12-1-1969.

Inicializar sin valor. Se muestra además el método `description()`.

In [4]:
fecha = qcf.QCDate()
print fecha.description(True)
print fecha.description(False)

12-01-1969
1969-01-12


Inicializar con una fecha específica. En este caso, el contructor realiza una validación de los parámetros iniciales.

In [5]:
fecha1= qcf.QCDate(14, 9, 2018) # día, mes, año
print fecha1.description(True)
print fecha1

14-09-2018
14-9-2018


In [6]:
try:
    fecha0 = qcf.QCDate(31, 2, 2018) # ¡¡¡ 31 de febrero !!!
except ValueError as e:
    print e

Invalid day for month = 2


In [7]:
# Métodos: set_day, set_month, set_year
fecha1.set_day(17)
fecha1.set_month(10)
fecha1.set_year(2019)
print fecha1.description(True)

17-10-2019


In [8]:
# Métodos day, month, year, weekday
print fecha1.day()
print fecha1.month()
print fecha1.year()
dia_semana = fecha1.week_day() # Retorna un enum QC_Financial.WeekDay 
print type(dia_semana)
print dia_semana

17
10
2019
<class 'QC_Financial.WeekDay'>
THU


In [9]:
# Método add_months
fecha2 = fecha1.add_months(1) # Suma n meses a fecha1 sin cambiar el valor de fecha1
print fecha1.description(True)
print fecha2.description(True)

17-10-2019
17-11-2019


In [10]:
# Método add_days
fecha3 = fecha1.add_days(30) # Suma n días a fecha1 sin cambiar el valor de fecha1
print fecha1.description(True)
print fecha3.description(True)

17-10-2019
16-11-2019


In [11]:
# Método day_diff
print fecha1.day_diff(fecha3) # Si fecha3 > fecha1 entonces el resultado es positivo
print fecha3.day_diff(fecha1)

30
-30


In [12]:
# Las fechas pueden compararse entre sí
print fecha1 == fecha2
print fecha1 != fecha2
print fecha1 < fecha2
print fecha1 <= fecha2
print fecha1 > fecha2
print fecha1 >= fecha2

False
True
True
True
False
False


In [13]:
# Un objeto QCDate es hashable, por lo tanto puede usarse como llave en un dict.
# El hash que retorna coincide con la representación como entero de Excel.
print fecha1.__hash__()

43755


In [14]:
# Convenience method que permite inicializar un QCDate a partir de un string.
# El formato del string debe ser yyyy&mm&dd donde & es un separador arbitrario.
fecha4 = qcf.build_qcdate_from_string('2020-01-01')
print fecha4.description(True)
fecha4 = qcf.build_qcdate_from_string('2020/01/02')
print fecha4.description(True)
fecha4 = qcf.build_qcdate_from_string('2020&01&03')
print fecha4.description(True)

01-01-2020
02-01-2020
03-01-2020


## Calendarios
Los calendarios se representan con objetos de tipo `BusinesssCalendar` y son **listas** de fechas arbitrarias que representan días feriados en alguna ciudad, país, región o unión de las anteriores. Para dar de alta un calendario se requiere una fecha inicial (`QCDate` y un número entero positivo que representa el plazo inicial total del calendario en años). El objeto `BusinessCalendar` incluye explícitamente todos los días 1 de enero y considera siempre como feriado los días sábado y domingo (aunque no pertenecen de forma explícita a la **lista**).   

In [15]:
scl = qcf.BusinessCalendar(fecha1, 10)
for holiday in scl.get_holydays():
    print holiday

1-1-2019
1-1-2020
1-1-2021
1-1-2022
1-1-2023
1-1-2024
1-1-2025
1-1-2026
1-1-2027
1-1-2028
1-1-2029


In [16]:
# Método add_holiday. Agrega una fecha a la lista.
scl.add_holiday(qcf.QCDate(2, 1, 2018))
for holiday in scl.get_holydays():
    print holiday

2-1-2018
1-1-2019
1-1-2020
1-1-2021
1-1-2022
1-1-2023
1-1-2024
1-1-2025
1-1-2026
1-1-2027
1-1-2028
1-1-2029


In [17]:
# Método next_busy_day. Dada una fecha, si ésta es hábil retorna la misma fecha,
## si, por el contrario, la fecha es inhábil, devuelve la siguiente fecha hábil del calendario.
print 'next:', scl.next_busy_day(qcf.QCDate(15, 9, 2018)) # es sábado
print 'Se agrega el 17-9-2018 a la lista'
scl.add_holiday(qcf.QCDate(17, 9, 2018))
print 'nuevo next:', scl.next_busy_day(qcf.QCDate(15, 9, 2018))

next: 17-9-2018
Se agrega el 17-9-2018 a la lista
nuevo next: 18-9-2018


In [18]:
# Método mod_next_busy_day. Opera igual que la función anterior a menos que la función anterior retorne una fecha
# del mes siguiente, en ese caso retorna la fecha hábil anterior.
print 'fecha: 2018-04-30'
abril30 = qcf.QCDate(30, 4, 2018)
scl.add_holiday(abril30)
print 'next:', scl.next_busy_day(abril30)
print 'mod next:', scl.mod_next_busy_day(qcf.QCDate(30, 4, 2018))
print 'abril30:', abril30

fecha: 2018-04-30
next: 1-5-2018
mod next: 27-4-2018
abril30: 30-4-2018


In [19]:
# Método prev_busy_day. Opera de forma análoga a mod_busy_day, pero retornando la fecha hábil anterior.
print 'prev:', scl.prev_busy_day(abril30)

prev: 27-4-2018


In [20]:
# Método shift. Suma un número n de días hábiles a una fecha.
mayo2 = qcf.QCDate(2, 5, 2018)
print scl.shift(mayo2, -1)
print scl.shift(abril30, 0)
print scl.shift(abril30, 1)
print scl.shift(abril30, 5)

1-5-2018
30-4-2018
1-5-2018
7-5-2018


## Fracciones de Año

In [21]:
# Existen Act/360, Act/365, 30/360 y Act/30
act360 = qcf.QCAct360()
act365 = qcf.QCAct365()
t30360 = qcf.QC30360()
act30 = qcf.QCAct30()
yfs = [act360, act365, t30360, act30]

In [22]:
# Métodos yf (sobrecargado) y count_days
for yf in yfs:
    print yf.yf(30)
    print yf.yf(fecha1, fecha3)
    print yf.count_days(fecha1, fecha3)
    print

0.0833333333333
0.0833333333333
30

0.0821917808219
0.0821917808219
30

0.0833333333333
0.0805555555556
29

1.0
1.0
30



## Factores de Capitalización

In [23]:
# Existen: (yf es la fracción de año asociada a la tasa de valor r)
#    QCLinearWf     ---> (1 + r * yf)
#    QCCompundWf    ---> (1 + r)**yf
#    QCContinousWf  ---> exp(r * yf)
lin_wf = qcf.QCLinearWf()
com_wf = qcf.QCCompoundWf()
exp_wf = qcf.QCContinousWf()

## Tasas de Interés

In [24]:
# Dar de alta una tasa de interés
tasa_lin_act360 = qcf.QCInterestRate(.1, act360, lin_wf)
tasa_com_act365 = qcf.QCInterestRate(.1, act365, com_wf)
tasas = [tasa_lin_act360, tasa_com_act365]

In [25]:
# Métodos get_value, set_value, wf (sobrecargado), dwf (sobrecargado)
for tasa in tasas:
    print 'get', tasa.get_value()
    tasa.set_value(.12)
    print 'get nuevo valor', tasa.get_value()
    print 'wf', tasa.wf(fecha1, fecha3)   # Retorna el factor de capitalización entre las fechas
    print 'dwf', tasa.dwf(fecha1, fecha3) # Retorna la derivada del factor de capitalización respecto al valor de la tasa
                                          # entre las fechas
    print 'wf1', tasa.wf(365)             # Retorna el factor de capitalización para el número de días
    print 'dwf1', tasa.dwf(365)           # Retorna la derivada del factor de capitalización respecto al valor de la tasa
                                          # para el número de días
    print

get 0.1
get nuevo valor 0.12
wf 1.01
dwf 0.0833333333333
wf1 1.12166666667
dwf1 1.01388888889

get 0.1
get nuevo valor 0.12
wf 1.00935820317
dwf 0.0740722751834
wf1 1.12
dwf1 1.0



### Tenor
Es una clase que representa el concepto de tenor (1D, 1M, 1Y ...).

In [26]:
# Ejemplo de Tenor
one_d = qcf.Tenor('1d')
one_m = qcf.Tenor('1M')
one_y = qcf.Tenor('1y')
t1d1m1y = qcf.Tenor('1D1M1Y')
t2Y3M = qcf.Tenor('2yadv3m')
tenors = [one_d, one_m, one_y, t1d1m1y, t2Y3M]

In [27]:
# Métodos get_string, get_years, get_months, get_years
for tenor in tenors:
    print 'string:', tenor.get_string()
    print 'dias:', tenor.get_days()
    print 'meses:', tenor.get_months()
    print 'años:', tenor.get_years()
    print

# Método set_tenor
t2Y3M.set_tenor('1m32y')
print t2Y3M.get_string()

string: 1D
dias: 1
meses: 0
años: 0

string: 1M
dias: 0
meses: 1
años: 0

string: 1Y
dias: 0
meses: 0
años: 1

string: 1Y1M1D
dias: 1
meses: 1
años: 1

string: 2Y3M
dias: 0
meses: 3
años: 2

32Y1M


## FX Rate
Es una clase que representa el concepto de tipo de cambio entre dos monedas. Para dar de alta un FXRate se requiere:

- QCCurrency: la moneda fuerte del par.
- QCCurrency: la moneda débl del par.

In [28]:
# Ejemplo. USDCLP. Método get_code. Se retorna el código del par según la convención usual.
usdclp = qcf.FXRate(usd, clp)
print 'código:', usdclp.get_code()

código: USDCLP


## Índices
Todo índice pertenece a un asset class. Los distintos asset class se representan con un el enum AssetClass. Los asset class disponibles son:

- InterestRate
- InterestRateVol
- Fx
- FxVol
- Equity
- EquityVol
- Commodity
- CommodityVol
- Credit
- CreditVol

In [29]:
# AssetClass. Existen
print qcf.AssetClass.InterestRate
print qcf.AssetClass.Fx

InterestRate
Fx


Para dar de alta un FXRateIndex se requiere:

- `FXRate`: el FXRate correspoondiente.
- `str`: nombre del índice
- `Tenor`: la regla de fixing, es 1D como el USD Observado o es 0D como un ínidce de cierre de día.
- `Tenor`: la regla para la valuta. Es 1D como el USDCLP o 2D como el EURUSD.
- `BusinessCalendar`: el calendario adecuado para aplicar las reglas de fixing y valuta.


In [30]:
# Ejemplo.
usdclp_obs = qcf.FXRateIndex(usdclp, 'USDOBS', one_d, one_d, scl)
print 'fecha de publicación:', fecha1.week_day(), fecha1
print 'fecha de fixing:', usdclp_obs.fixing_date(fecha1)
print 'fecha de valuta:', usdclp_obs.value_date(fecha1)

fecha de publicación: THU 17-10-2019
fecha de fixing: 16-10-2019
fecha de valuta: 17-10-2019


## Cashflows

### Simple Cashflow
Un objeto de tipo `SimpleCashflow` representa un flujo de caja cierto en una fecha y moneda.

In [31]:
# Como se inicializa un objeto SimpleCashflow.
simple_cashflow = qcf.SimpleCashflow(fecha1, # fecha del flujo
                                    100,     # monto
                                    clp)     # moneda

In [32]:
# Métodos amount, ccy y date
print simple_cashflow.amount()
print simple_cashflow.ccy().get_iso_code()
print simple_cashflow.date()

100.0
CLP
17-10-2019


In [33]:
# Función show. Envuelve de forma cómoda todo el flujo en una tupla.
# La función show está sobregargada y admite muchos tipos de flujo.
print qcf.show(simple_cashflow)

('2019-10-17', 100.0, 'CLP')


### Simple Multicurrency Cashflow
Un objeto de tipo `SimpleMultiCurrencyCashflow` representa un flujo de caja cierto en una fecha y moneda, que, sin embargo, se liquidará en una segunda moneda utilizando el valor de un índice de tipo de cambio. Para dar de alta uno de estos objetos se requiere:

- `QCDate`: fecha final
- `float`: nominal
- `QCCurrency`: la moneda del nominal
- `QCDate`: la fecha de publicación del `FXRateIndex` que se usará en la conversión
- `QCCurrency`: la moneda final del flujo
- `FXRateIndex`: el índice de tipo de cambio a utilizar
- `float`: el valor del índice de tipo de cambio

In [34]:
# Ejemplo.
simple_mccy_cashflow = qcf.SimpleMultiCurrencyCashflow(fecha3,
                                                      700.00,
                                                      clp,
                                                      fecha3,
                                                      usd,
                                                      usdclp_obs,
                                                      700.00)
print 'fecha flujo:', simple_mccy_cashflow.date()
print 'nominal:', simple_mccy_cashflow.nominal()
print 'moneda nominal:', simple_mccy_cashflow.ccy().get_iso_code()
print 'flujo:', simple_mccy_cashflow.amount()
print 'moneda flujo:', simple_mccy_cashflow.settlement_ccy().get_iso_code()


fecha flujo: 16-11-2019
nominal: 700.0
moneda nominal: CLP
flujo: 1.0
moneda flujo: USD


In [35]:
# El valor del índice se puede alterar (existe un setter).
simple_mccy_cashflow.set_fx_rate_index_value(800.00)
print 'flujo:', simple_mccy_cashflow.amount()

flujo: 0.875


In [36]:
# Ejemplo 2. Las monedas están invertidas respecto al caso anterior. El valor del índice se usa de la forma correcta.
simple_mccy_cashflow = qcf.SimpleMultiCurrencyCashflow(fecha3,
                                                      1,
                                                      usd,
                                                      fecha3,
                                                      clp,
                                                      usdclp_obs,
                                                      700.00)
print 'fecha flujo:', simple_mccy_cashflow.date()
print 'nominal:', simple_mccy_cashflow.nominal()
print 'moneda nominal:', simple_mccy_cashflow.ccy().get_iso_code()
print 'flujo:', simple_mccy_cashflow.amount()
print 'moneda flujo:', simple_mccy_cashflow.settlement_ccy().get_iso_code()

fecha flujo: 16-11-2019
nominal: 1.0
moneda nominal: USD
flujo: 700.0
moneda flujo: CLP


In [37]:
# Ejemplo 3. Hay inconsistencia entre las monedas y el tipo de cambio del índice.
eur = qcf.QCEUR()
try:
    simple_mccy_cashflow = qcf.SimpleMultiCurrencyCashflow(fecha3,
                                                           1,
                                                           usd,
                                                           fecha3,
                                                           eur,
                                                           usdclp_obs,
                                                           700.00)
except ValueError as e:
    print 'Error:', e

Error: Fx Rate Index provided is not compatible with nominal and settlement currency. 


In [38]:
# Función show
qcf.show(simple_mccy_cashflow)

('2019-11-16', 1.0, 'USD', '2019-11-16', 'CLP', 'USDOBS', 700.0, 700.0)

### Fixed Rate Cashflow
Un objeto de tipo `FixedRateCashflow` representa un flujo de caja calculado a partir de la aplicación de una tasa prefijada, entre dos fechas prefijadas a un nominal prefijado. Para dar de alta uno de estos objetos se requiere:

- `QCDate`: fecha inicio (para la aplicación de la tasa)
- `QCDate`: fecha final (para la aplicación de la tasa)
- `QCDate`: fecha de pago
- `float`: nominal (monto al que se le aplica la tasa)
- `float`: amortización (eventual flujo de caja que corresponde a una porción del nominal)
- `bool`: indica si la amortización anterior es un flujo de caja o sólo una disminución de nominal
- `QCInterestRate`: la tasa de interés a aplicar (su valor y convenciones)
- `QCCurrency`: moneda del nominal y del flujo de caja

In [39]:
# Ejemplo
fecha_inicio = qcf.QCDate(20, 9, 2018)
fecha_final = qcf.QCDate(20, 9, 2019)
fecha_pago = qcf.QCDate(23, 9, 2019)
tasa = qcf.QCInterestRate(.03, act360, lin_wf)
fixed_rate_cashflow = qcf.FixedRateCashflow(fecha_inicio,
                                           fecha_final,
                                           fecha_pago,
                                           1000000000.0,
                                           100000000.0,
                                           True,
                                           tasa,
                                           clp)

In [40]:
# Getters
# TODO: get_rate
print "Fecha Inicio:", fixed_rate_cashflow.get_start_date()
print "Fecha Final:", fixed_rate_cashflow.get_end_date()
print "Fecha Pago:", fixed_rate_cashflow.get_settlement_date()
print "Moneda:", fixed_rate_cashflow.ccy().get_iso_code()
print "Nominal:", fixed_rate_cashflow.get_nominal()
print "Amortización:", fixed_rate_cashflow.get_amortization()

Fecha Inicio: 20-9-2018
Fecha Final: 20-9-2019
Fecha Pago: 23-9-2019
Moneda: CLP
Nominal: 1000000000.0
Amortización: 100000000.0


In [41]:
# Setters
# TODO: set_rate
nuevo_nominal = 2000000000.0
fixed_rate_cashflow.set_nominal(nuevo_nominal)
print "Nuevo nominal:", fixed_rate_cashflow.get_nominal()

nueva_amortizacion = 200000000.0
fixed_rate_cashflow.set_amortization(nueva_amortizacion)
print "Nueva amortización:", fixed_rate_cashflow.get_amortization()


Nuevo nominal: 2000000000.0
Nueva amortización: 200000000.0


In [42]:
# Cálculos
print "Flujo Total:", fixed_rate_cashflow.amount()
print "Check:", fixed_rate_cashflow.get_nominal() * .03 * 365.0/360 + fixed_rate_cashflow.get_amortization()
print
fecha_intermedia = qcf.QCDate(2, 1, 2019)
print "Interés Devengado:", fixed_rate_cashflow.accrued_interest(fecha_intermedia)
print "Check:",  fixed_rate_cashflow.get_nominal() * .03 * fecha_inicio.day_diff(fecha_intermedia) /360.0

Flujo Total: 260833333.333
Check: 260833333.333

Interés Devengado: 17333333.3333
Check: 17333333.3333


In [43]:
# Función show
print qcf.show(fixed_rate_cashflow)

('2018-09-20', '2019-09-20', '2019-09-23', 2000000000.0, 200000000.0, 60833333.333333515, True, 260833333.33333352, 'CLP', 0.03, 'LinAct360')


### Fixed Rate Multi Currency Cashflow
Un objeto de tipo `FixedRateMultiCurrencyCashflow` representa un flujo de caja a tasa fija (`FixedRateCashflow`) que se liquidará en una moneda distinta de la moneda del nominal utilizando el valor a una cierta fecha de un índice de tipo de cambio prefijado. Para dar de alta uno de estos objetos se requiere:

- `QCDate`: fecha inicio (para la aplicación de la tasa)
- `QCDate`: fecha final (para la aplicación de la tasa)
- `QCDate`: fecha de pago
- `float`: nominal (monto al que se le aplica la tasa)
- `float`: amortización (eventual flujo de caja que corresponde a una porción del nominal)
- `bool`: indica si la amortización anterior es un flujo de caja o sólo una disminución de nominal
- `QCInterestRate`: la tasa de interés a aplicar (su valor y convenciones)
- `QCCurrency`: moneda del nominal
- `QCDate`: fecha de publicación del índice de tipo de cambio
- `QCCurrency`: moneda en la que se liquida el flujo
- `FXRateIndex`: índice de tipo de cambio a utilizar
- `float`: valor del índice de tipo de cambio

In [44]:
# Ejemplo
fecha_inicio = qcf.QCDate(20, 9, 2018)
fecha_final = qcf.QCDate(20, 9, 2019)
fecha_pago = qcf.QCDate(23, 9, 2019)
fecha_publicacion = qcf.QCDate(23, 9, 2019)
usd = qcf.QCUSD()
indice = usdclp_obs
valor_indice = 700.0
nominal = 700000000.0
amort = 70000000
fixed_rate_mccy_cashflow = qcf.FixedRateMultiCurrencyCashflow(fecha_inicio,
                                                              fecha_final,
                                                              fecha_pago,
                                                              nominal,
                                                              amort,
                                                              False,
                                                              tasa,
                                                              clp,
                                                              fecha_publicacion,
                                                              usd,
                                                              indice,
                                                              valor_indice)
print fixed_rate_mccy_cashflow

**TODO: get_rate.** Este getter no debe ser un getter tradicional, ya que no es necesario que retorne una referencia a todo el objeto ``QCInterestRate``, basta con el valor y la descripción de yf y wf asociado (mejor llamarlo get_rate_info).

In [45]:
# Getters
print "Fecha Inicio:", fixed_rate_mccy_cashflow.get_start_date()
print "Fecha Final:", fixed_rate_mccy_cashflow.get_end_date()
print "Fecha Pago:", fixed_rate_mccy_cashflow.get_settlement_date()
print "Moneda del Nominal:", fixed_rate_mccy_cashflow.ccy()
print "Nominal:", fixed_rate_mccy_cashflow.get_nominal()
print "Amortización:", fixed_rate_mccy_cashflow.get_amortization()
print "Moneda de Liquidación:", fixed_rate_mccy_cashflow.settlement_currency()

Fecha Inicio: 20-9-2018
Fecha Final: 20-9-2019
Fecha Pago: 23-9-2019
Moneda del Nominal: CLP
Nominal: 700000000.0
Amortización: 70000000.0
Moneda de Liquidación: USD


**TODO: set_rate_value.** Debe establecer el valor de la tasa de interés.

In [46]:
# Setters
nuevo_nominal = 100.0
fixed_rate_mccy_cashflow.set_nominal(nuevo_nominal)
print "Nuevo nominal:", fixed_rate_mccy_cashflow.get_nominal()

nueva_amortizacion = 10.0
fixed_rate_mccy_cashflow.set_amortization(nueva_amortizacion)
print "Nueva amortización:", fixed_rate_mccy_cashflow.get_amortization()


Nuevo nominal: 100.0
Nueva amortización: 10.0


In [47]:
# Cálculos
fixed_rate_mccy_cashflow.set_nominal(nominal)
fixed_rate_mccy_cashflow.set_amortization(amort)
print "Flujo Total:", fixed_rate_mccy_cashflow.amount()
print "Check:", (fixed_rate_mccy_cashflow.get_nominal() * .03 * 365.0/360) / valor_indice
print
fecha_intermedia = qcf.QCDate(2, 1, 2019)
print "Interés Devengado:", fixed_rate_mccy_cashflow.accrued_interest(fecha_intermedia)
print "Check:",  fixed_rate_mccy_cashflow.get_nominal() * .03 * fecha_inicio.day_diff(fecha_intermedia) /360.0

Flujo Total: 30416.6666667
Check: 30416.6666667

Interés Devengado: 6066666.66667
Check: 6066666.66667


In [48]:
# Función show
print qcf.show(fixed_rate_mccy_cashflow)

('2018-09-20', '2019-09-20', '2019-09-23', 700000000.0, 70000000.0, 21291666.66666673, False, 30416.66666666676, 'CLP', 0.03, 'LinAct360', '2019-09-23', 'USD', 'USDOBS', 700.0, 100000.0, 30416.66666666676)


### Ibor Cashflow
Un objeto de tipo `IborCashflow` representa un flujo de caja calculado a partir de la aplicación de una tasa flotante fijada en una cierta fecha (Libor, Euribor, TAB, ...) , entre dos fechas prefijadas a un nominal prefijado. Para dar de alta uno de estos objetos se requiere:

- `InterestRateIndex`: el índice de tasa de interés prefijado
- `QCDate`: fecha inicio (para la aplicación de la tasa)
- `QCDate`: fecha final (para la aplicación de la tasa)
- `QCDate`: fecha de fijación del índice de tasa de interés 
- `QCDate`: fecha de pago
- `float`: nominal (monto al que se le aplica la tasa)
- `float`: amortización (eventual flujo de caja que corresponde a una porción del nominal)
- `bool`: indica si la amortización anterior es un flujo de caja o sólo una disminución de nominal
- `QCCurrency`: moneda del nominal y del flujo de caja
- `float`: spread aditivo a aplicar a la fijación del índice
- `float`: spread multiplicativo o gearing a aplicar a la fijación del índice

Para dar de alta un `InterestRateIndex` se requiere:

- `str`: código del índice
- `QCInterestRate`: un objeto tasa de interés que contenga las convenciones del índice
- `Tenor`: el lag de inicio del índice respecto a la fecha de fixing (por ejemplo 2d para Libor USD)
- `Tenor`: el tenor del índice (3M por ejemplo para Libor USD 3M)
- `QCBusinessCalendar`: el calendario de fixing
- `QCBusinessCalendar`: el calendario de pago
- `QCCurrency`: la moneda a la que corresponde el índice (por ejemplo EUR para EURIBOR 3M)


**TODO: agregar end_date_adjustment al objeto (FOLLOW o MOD FOLLOW).** Esto debiera tener un correspondiente cambio en la BBDD de Front Desk.

In [49]:
# Se define el índice
codigo = 'LIBORUSD3M'
lin_act360 = qcf.QCInterestRate(.0, act360, lin_wf)
fixing_lag = qcf.Tenor('2d')
tenor = qcf.Tenor('3m')
fixing_calendar = scl     # No es el calendario correcto, pero sirve para el ejemplo
settlement_calendar = scl # Ídem arriba
libor_usd_3m = qcf.InterestRateIndex(codigo,
                                    lin_act360,
                                    fixing_lag,
                                    tenor,
                                    fixing_calendar,
                                    settlement_calendar,
                                    usd)

# Getters
print "Tenor:", libor_usd_3m.get_tenor()
print "Tasa:", libor_usd_3m.get_rate()
print

# Para construir un fixing en particular
libor_usd_3m.set_rate_value(.01)
print "Fixing Tasa:", libor_usd_3m.get_rate()
fecha_fixing = qcf.QCDate(20, 9, 2018)
print "Fecha Inicio:", libor_usd_3m.get_start_date(fecha_fixing)
print "Fecha Final:", libor_usd_3m.get_end_date(fecha_fixing)


Tenor: 3M
Tasa: 0.000000 Act360 Lin

Fixing Tasa: 0.010000 Act360 Lin
Fecha Inicio: 24-9-2018
Fecha Final: 24-12-2018


Con esto, veamos un ejemplo de construcción y uso de un `IborCashflow`.

**TODO:** Se debe crear un mecanismo de WARNING para las eventuales inconsistencias entre las fechas de inicio y fin del ``InterestRateIndex`` y las fechas de inicio y fin del ``IborCashflow``.

In [50]:
# Ejemplo
fecha_inicio = qcf.QCDate(20, 9, 2018)
fecha_final = qcf.QCDate(20, 9, 2019)
fecha_pago = qcf.QCDate(23, 9, 2019)
fecha_fixing = qcf.QCDate(20, 9, 2018)
nominal = 1000000.0
amort = 100000.0
spread = .0
gearing = 1.0
ibor_cashflow = qcf.IborCashflow(libor_usd_3m,
                                 fecha_inicio,
                                 fecha_final,
                                 fecha_fixing,
                                 fecha_pago,
                                 nominal,
                                 amort,
                                 True,
                                 usd,
                                 spread,
                                 gearing)

In [51]:
# Getters
print "Fecha Fixing:\t", ibor_cashflow.get_fixing_date()
print "Fecha Inicio:\t", ibor_cashflow.get_start_date()
print "Fecha Final:\t", ibor_cashflow.get_end_date()
print "Fecha Pago:\t", ibor_cashflow.get_settlement_date()
print "Nominal:\t", ibor_cashflow.get_nominal()
print "Amortización:\t", ibor_cashflow.get_amortization()
print "Moneda:\t\t", ibor_cashflow.ccy()
print "Valor Tasa:\t", ibor_cashflow.get_interest_rate_value()

Fecha Fixing:	20-9-2018
Fecha Inicio:	20-9-2018
Fecha Final:	20-9-2019
Fecha Pago:	23-9-2019
Nominal:	1000000.0
Amortización:	100000.0
Moneda:		USD
Valor Tasa:	0.01


In [52]:
# Setters
nuevo_nominal = 2000000.0
ibor_cashflow.set_nominal(nuevo_nominal)
print "Nominal:\t", ibor_cashflow.get_nominal()

nueva_amortizacion = 200000.0
ibor_cashflow.set_amortization(nueva_amortizacion)
print "Amortización:\t", ibor_cashflow.get_amortization()

nuevo_valor_tasa = .02
ibor_cashflow.set_interest_rate_value(nuevo_valor_tasa)

Nominal:	2000000.0
Amortización:	200000.0


In [53]:
# Cálculos
print "Flujo:", ibor_cashflow.amount()

fecha_devengo = qcf.QCDate(20, 7, 2019)
print "Interés Devengado:", ibor_cashflow.accrued_interest(fecha_devengo)
tasa = ibor_cashflow.get_interest_rate_value()
print "Check:", tasa * fecha_inicio.day_diff(fecha_devengo) / 360.0 * ibor_cashflow.get_nominal()

Flujo: 240555.555556
Interés Devengado: 33666.6666667
Check: 33666.6666667


In [54]:
# Función show
print qcf.show(ibor_cashflow)

('2018-09-20', '2019-09-20', '2018-09-20', '2019-09-23', 2000000.0, 200000.0, 40555.55555555568, True, 240555.55555555568, 'USD', 'LIBORUSD3M', 0.02, 0.0, 1.0, 'LinAct360')


### Icp Clp Cashflow
Un objeto de tipo `IcpClpCashflow` representa un flujo de caja calculado como un cupón de la pata flotante de un swap ICP (cámara promedio) de Chile. Para dar de alta uno de estos objetos se requiere:

- `QCDate`: fecha inicio (para la aplicación de la tasa)
- `QCDate`: fecha final (para la aplicación de la tasa)
- `QCDate`: fecha de pago
- `float`: nominal (monto al que se le aplica la tasa)
- `float`: amortización (eventual flujo de caja que corresponde a una porción del nominal)
- `bool`: indica si la amortización anterior es un flujo de caja o sólo una disminución de nominal
- `float`: spread aditivo a aplicar a la fijación de la TNA
- `float`: spread multiplicativo o gearing a aplicar a la fijación de la TNA
- `float`: el valor del ICP a fecha de inicio (u otro valor arbitrario si el valor es desconocido)
- `float`: el valor del ICP a fecha final (u otro valor arbitrario si el valor es desconocido)

Recordar que TNA significa **Tasa Nominal Anual** y se determina utilizando los valores del índice ICP en la fecha de inicio y fecha final del `IcpClpCashflow`.

In [55]:
# Ejemplo
fecha_inicio = qcf.QCDate(20, 9, 2018)
fecha_final = qcf.QCDate(20, 9, 2019)
fecha_pago = qcf.QCDate(23, 9, 2019)
nominal = 1000000000.0
amort = 100000000.0
spread = 0.0
gearing = 1.0
icp_clp_cashflow = qcf.IcpClpCashflow(fecha_inicio,
                                      fecha_final,
                                      fecha_pago,
                                      nominal,
                                      amort,
                                      True,
                                      spread,
                                      gearing,
                                      10000.0,
                                      10250.0)

In [56]:
# Getters
print "Fecha Inicio:", icp_clp_cashflow.get_start_date()
print "Fecha Final:", icp_clp_cashflow.get_end_date()
print "ICP Fecha Inicio:", icp_clp_cashflow.get_start_date_icp()
print "ICP Fecha Final:", icp_clp_cashflow.get_end_date_icp()
print "Valor TNA Todo el Período:", icp_clp_cashflow.get_rate_value()
print "Check:", round((10250.0 / 10000 - 1) * 360.0 / fecha_inicio.day_diff(fecha_final), 4)
print "Nominal:", icp_clp_cashflow.get_nominal()
print "Amortización:", icp_clp_cashflow.get_amortization()
print "Tipo de Tasa:", icp_clp_cashflow.get_type_of_rate()
print "Moneda:", icp_clp_cashflow.ccy()

Fecha Inicio: 20-9-2018
Fecha Final: 20-9-2019
ICP Fecha Inicio: 10000.0
ICP Fecha Final: 10250.0
Valor TNA Todo el Período: 0.0247
Check: 0.0247
Nominal: 1000000000.0
Amortización: 100000000.0
Tipo de Tasa: LinAct360
Moneda: CLP


In [57]:
# Setters
decimales_para_tna = 6
icp_clp_cashflow.set_tna_decimal_places(decimales_para_tna)
print "Nueva TNA:", icp_clp_cashflow.get_rate_value()

nuevo_nominal = 100
icp_clp_cashflow.set_nominal(nuevo_nominal)
print "Nuevo Nominal:", icp_clp_cashflow.get_nominal()

nueva_amortizacion = 10
icp_clp_cashflow.set_amortization(nueva_amortizacion)
print "Nueva Amortización:", icp_clp_cashflow.get_amortization()

nuevo_icp_inicio = 20000
icp_clp_cashflow.set_start_date_icp(nuevo_icp_inicio)
print "Nuevo ICP Inicio:", icp_clp_cashflow.get_start_date_icp()

nuevo_icp_final = 20000
icp_clp_cashflow.set_end_date_icp(nuevo_icp_final)
print "Nuevo ICP Final:", icp_clp_cashflow.get_end_date_icp()
print "Check TNA Final:", icp_clp_cashflow.get_rate_value()

Nueva TNA: 0.024658
Nuevo Nominal: 100.0
Nueva Amortización: 10.0
Nuevo ICP Inicio: 20000.0
Nuevo ICP Final: 20000.0
Check TNA Final: 0.0


In [58]:
# Cálculos
decimales_para_tna = 4 # Se vuelve a 4 decimales de tasa
icp_clp_cashflow.set_tna_decimal_places(decimales_para_tna)

nuevo_icp_inicio = 10000.0
icp_clp_cashflow.set_start_date_icp(nuevo_icp_inicio)

nuevo_icp_final = 10250.0
icp_clp_cashflow.set_end_date_icp(nuevo_icp_final)

print "Flujo:", icp_clp_cashflow.amount()

fecha_devengo = qcf.QCDate(29, 3, 2019)
icp_devengo = 10125.0
tna_devengo = icp_clp_cashflow.get_tna(fecha_devengo, icp_devengo)
print "TNA fijada a " + fecha_devengo.description(True) + ":", tna_devengo
print "Check:", round((10125.0 / 10000.0 - 1) * 360.0 / fecha_inicio.day_diff(fecha_devengo), decimales_para_tna)
print "Interés Devengado a " + fecha_devengo.description(True) + ":", icp_clp_cashflow.accrued_interest(fecha_devengo, icp_devengo)
print "Check:", 100 * tna_devengo * fecha_inicio.day_diff(fecha_devengo) / 360.0

Flujo: 12.5043055556
TNA fijada a 29-03-2019: 0.0237
Check: 0.0237
Interés Devengado a 29-03-2019: 1.25083333333
Check: 1.25083333333


In [59]:
print qcf.show(icp_clp_cashflow)

('2018-09-20', '2019-09-20', '2019-09-23', 100.0, 10.0, True, 12.504305555555565, 'CLP', 10000.0, 10250.0, 0.0247, 2.504305555555564, 0.0, 1.0, 'LinAct360')


### Icp Clf Cashflow
Un objeto de tipo `IcpClfCashflow` representa un flujo de caja calculado como un cupón de la pata flotante de un swap ICP (cámara promedio) en UF de Chile. Para dar de alta uno de estos objetos se requiere:

- `QCDate`: fecha inicio (para la aplicación de la tasa)
- `QCDate`: fecha final (para la aplicación de la tasa)
- `QCDate`: fecha de pago
- `float`: nominal (monto al que se le aplica la tasa)
- `float`: amortización (eventual flujo de caja que corresponde a una porción del nominal)
- `bool`: indica si la amortización anterior es un flujo de caja o sólo una disminución de nominal
- `float`: spread aditivo a aplicar a la fijación de la TRA
- `float`: spread multiplicativo o gearing a aplicar a la fijación de la TRA
- `vector<float>`: objeto `double_vec` (en Python) que contiene ICP Inicio, ICP Final, UF Inicio, UF Final (se debe respetar el orden)

Recordar que TRA significa **Tasa Real Anual** y se determina utilizando los valores del índice ICP y los valores de la UF en la fecha de inicio y fecha final del `IcpClfCashflow`.

In [60]:
# Ejemplo
fecha_inicio = qcf.QCDate(20, 9, 2018)
fecha_final = qcf.QCDate(20, 9, 2019)
fecha_pago = qcf.QCDate(23, 9, 2019)
nominal = 300000.0
amort = 100000.0
spread = 0.0
gearing = 1.0
icp_uf = qcf.double_vec()
icp_uf.append(10000.0)
icp_uf.append(10250.0)
icp_uf.append(10000.0)
icp_uf.append(10400.0)
icp_clf_cashflow = qcf.IcpClfCashflow(fecha_inicio,
                                      fecha_final,
                                      fecha_pago,
                                      nominal,
                                      amort,
                                      True,
                                      spread,
                                      gearing,
                                      icp_uf)

In [61]:
# Getters
print "Fecha Inicio:", icp_clf_cashflow.get_start_date()
print "Fecha Final:", icp_clf_cashflow.get_end_date()
print "ICP Fecha Inicio:", icp_clf_cashflow.get_start_date_icp()
print "ICP Fecha Final:", icp_clf_cashflow.get_end_date_icp()
print "UF Fecha Inicio:", icp_clf_cashflow.get_start_date_uf()
print "UF Fecha Final:", icp_clf_cashflow.get_end_date_uf()
print "Valor TRA Todo el Período:", icp_clf_cashflow.get_rate_value()
tna = icp_clf_cashflow.get_tna(fecha_final, 10250.0)
dias = fecha_inicio.day_diff(fecha_final)
tra = ((1 + tna * dias / 360.0) * 10000.0 / 10400.0 - 1) * 360.0 / dias
print "Check TRA:", round(tra, 6)
print "Nominal:", icp_clf_cashflow.get_nominal()
print "Amortización:", icp_clf_cashflow.get_amortization()
print "Tipo de Tasa:", icp_clf_cashflow.get_type_of_rate()
print "Moneda:", icp_clf_cashflow.ccy()

Fecha Inicio: 20-9-2018
Fecha Final: 20-9-2019
ICP Fecha Inicio: 10000.0
ICP Fecha Final: 10250.0
UF Fecha Inicio: 10000.0
UF Fecha Final: 10400.0
Valor TRA Todo el Período: -0.014185
Check TRA: -0.014185
Nominal: 300000.0
Amortización: 100000.0
Tipo de Tasa: LinAct360
Moneda: CLF


In [62]:
# Setters
decimales_para_tra = 8
icp_clf_cashflow.set_tra_decimal_places(decimales_para_tra)
print "Nueva TRA:", icp_clf_cashflow.get_rate_value()

nuevo_nominal = 100000.0
icp_clf_cashflow.set_nominal(nuevo_nominal)
print "Nuevo Nominal:", icp_clf_cashflow.get_nominal()

nueva_amortizacion = 10000.0
icp_clf_cashflow.set_amortization(nueva_amortizacion)
print "Nueva Amortización:", icp_clf_cashflow.get_amortization()

nuevo_icp_inicio = 20000.0
icp_clf_cashflow.set_start_date_icp(nuevo_icp_inicio)
print "Nuevo ICP Inicio:", icp_clf_cashflow.get_start_date_icp()

nuevo_icp_final = 20000.0
icp_clf_cashflow.set_end_date_icp(nuevo_icp_final)
print "Nuevo ICP Final:", icp_clf_cashflow.get_end_date_icp()
print "Check TNA Final:", icp_clf_cashflow.get_tna(fecha_final, nuevo_icp_final)

Nueva TRA: -0.01418467
Nuevo Nominal: 100000.0
Nueva Amortización: 10000.0
Nuevo ICP Inicio: 20000.0
Nuevo ICP Final: 20000.0
Check TNA Final: 0.0
